# Object Recognition Dataset Demo

This notebook is an example of how to load a dataset for use with Object Recognition. This dataset contains bounding boxes of all the classes.

This demo will load the dataset using the Abyss Deep Learning libraries, and display an example image with the bounding boxes overlayed.

In [1]:
# Imports

import json
import math
import os

from keras.backend import clear_session
import matplotlib.pyplot as plt
import numpy as np

from abyss_deep_learning.datasets.coco import ImageClassificationDataset
from abyss_deep_learning.datasets.translators import CocoCaptionTranslator
from abyss_deep_learning.keras.classification import caption_map_gen, onehot_gen, hamming_loss
from abyss_deep_learning.keras.utils import lambda_gen, batching_gen
from abyss_deep_learning.keras.models import ImageClassifier
#from abyss_deep_learning.utils import balanced_set

from matplotlib.collections import PatchCollection


import matplotlib.pyplot as plt
import matplotlib.patches as patches

Using TensorFlow backend.


## To Go Into deep-learning/abyss_deep_learning/datasets/coco.py

The classes in the cell below should go into deep-learning/abyss_deep_learning/datasets/coco.py once they have been reviewed. They are the standard interface to an Object Recognition dataset.

The imports can be skipped.

! Remove once it has been added !

In [2]:
import numpy as np

import json
from collections import Counter
from contextlib import redirect_stdout
from sys import stderr
import concurrent.futures
import itertools
import os
import random
import sys

from skimage.io import imread

# These imports can be skipped
from abyss_deep_learning.base.datasets import DatasetTaskBase, DatasetTypeBase
from abyss_deep_learning.datasets.coco import CocoInterface, CocoDataset, ImageDatatype
from abyss_deep_learning.datasets.translators import AnnotationTranslator

from abyss_deep_learning.datasets.coco import _noop

class ObjectRecognitionTask(CocoInterface, DatasetTaskBase):
    def __init__(self, coco, translator=None, num_classes=None, **kwargs):
        """
        Segmentation arguments:
            coco (pycocotools.COCO): The COCO object to read the targes from
            translator (AnnotationTranslator, optional): An instance of an abyss_deep_learning.datasets.translators.AnnotationTranslator
            num_classes (int, optional): The number of classes to generate data for; if None then infer from coco.cats
            cached (bool, optional): Whether to cache the entire dataset into memory.
        """
        CocoInterface.__init__(self, coco, **kwargs)
        assert isinstance(translator, (AnnotationTranslator, type(None)))
        self.translator = translator or AnnotationTranslator()
        self.num_classes = num_classes if num_classes else len(self.coco.cats) + 1
        self.stats = dict()
        self._targets = dict()

        self._preprocess_targets = kwargs.get('preprocess_targets', _noop)

        if kwargs.get('cached', False):
            with concurrent.futures.ProcessPoolExecutor() as executor:
                for data_id, targets in zip(
                        self.data_ids, executor.map(self.load_targets, self.data_ids)):
                    self._targets[data_id] = targets
        # self._calc_class_stats()

    def load_targets(self, data_id, **kwargs):
        # assert np.issubdtype(type(data_id), np.integer), "Must pass exactly one ID"
        if data_id in self._targets:
            return self._targets[data_id]
        img = self.coco.loadImgs(ids=[data_id])[0]
        anns = [self.translator.translate(ann) for ann in self.coco.loadAnns(
            self.coco.getAnnIds([data_id])) if self.translator.filter(ann)]
        if anns:
            boxes = np.array([ann['bbox'] for ann in anns])
            class_ids = np.array([ann['category_id'] for ann in anns])
            return self._preprocess_targets(class_ids, boxes)
        return None  # TODO check

    def _calc_class_stats(self):
        if not self.stats:
            self.stats = dict()
            class_count = dict()
            for data_id in self.data_ids:
                target = self.load_targets(data_id)[0].argmax(-1)  # [0] is the category_id
                for key, val in Counter(target.ravel().tolist()).items():
                    class_count[key] = class_count.get(key, 0) + val

            self.stats['class_weights'] = np.array(
                [class_count.get(key, 0) for key in range(self.num_classes)], dtype=np.float64)
            self.stats['class_weights'] **= -1.0
            self.stats['class_weights'] /= self.stats['class_weights'].min()

    @property
    def class_weights(self):
        '''Returns the class weights that will balance the backprop update over the class distribution.'''
        return self.stats['class_weights']

    def print_class_stats(self):
        '''Prints statistics about the class/image distribution.'''
        self._calc_class_stats()
        print("{:s} class stats {:s}".format('=' * 8, '=' * 8))
        print("class weights:")
        print(" ", self.class_weights)


class ImageObjectRecognitionDataset(CocoDataset, ImageDatatype, ObjectRecognitionTask):
    # TODO:
    #   *  Class statistics readout
    #   *  Support for computing class weights given current dataset config
    #   *  Support for forcing class balance by selecting IDs evenly
    #   *  Generator data order optimization
    #   *  Support for visualising data sample or prediction with same format
    def __init__(self, json_path, **kwargs):
        CocoDataset.__init__(self, json_path, **kwargs)
        ImageDatatype.__init__(self, self.coco, **kwargs)
        ObjectRecognitionTask.__init__(self, self.coco, **kwargs)

    def sample(self, image_id=None, **kwargs):
        if not image_id:
            image_id = random.choice(self.data_ids)
        return (self.load_data(image_id, **kwargs), self.load_targets(image_id, **kwargs))

    def generator(self, data_ids=None, shuffle_ids=False, endless=False, **kwargs):
        if not data_ids:
            data_ids = list(self.data_ids)
        if shuffle_ids:
            random.shuffle(data_ids)
        iterator = itertools.cycle if endless else iter
        for data_id in iterator(data_ids):
            yield self.load_data(data_id, **kwargs), self.load_targets(data_id, **kwargs)


## To go into deep-learning/abyss_deep_learning/datasets/translators.py

The classes in the cell below should be added to deep-learning/abyss_deep_learning/datasets/translators.py as they perform functions necessary for an Object Recognition dataset.

NEED TO REVIEW - what translator to use - needs to be matched with the ObjectRecognitionTask in coco.py

! Remove once it has been added !

In [3]:
from abyss_deep_learning.datasets.translators import AnnotationTranslator

class CocoCategoryTranslator(AnnotationTranslator):
    def filter(self, annotation):
        '''Filter out non-caption annotations'''
        return True

    def translate(self, annotation):
        '''Return a list of strings'''
        return [annotation['category_id']]

class CocoObjectTranslator(AnnotationTranslator):
    def filter(self, annotation):
        '''Filter out non-caption annotations'''
        return True

    def translate(self, annotation):
        '''Return annotation and bounding box'''
        return [annotation['category_id'], annotation['bbox']]


class CocoAllTranslator(AnnotationTranslator):
    def filter(self, annotation):
        '''Filter out non-caption annotations'''
        return True

    def translate(self, annotation):
        '''Returns the entire coco annotation dictionary for the specified annId'''
        return annotation

## Loading the dataset

Enter the dataset you wish to use here.

The complete dataset used for this example is created by:
```python
os.path.join(DATA_DIR, TRAIN_JSON)

In [4]:
DATA_DIR = "/mnt/ssd_x/simbuoy/datasets/"
TRAIN_JSON = "train1/train-sim.json"
VAL_JSON = "val1/val-sim.json"

train_ds = ImageObjectRecognitionDataset(os.path.join(DATA_DIR, TRAIN_JSON),image_dir=DATA_DIR,cached=False,translator=CocoAllTranslator())

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## Print the Class Statistics

In [5]:
catIds = train_ds.coco.getCatIds()
train_ds.print_class_stats()

======== class stats ========
class weights:
  [ 1.          6.16666667  3.36363636  4.625       9.25        7.4
         inf 37.                 inf 12.33333333]


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:71: RuntimeWarning: divide by zero encountered in reciprocal


## Sample from the Dataset

The following cell samples from the dataset, and grabs a single image and its annotations.

The annotations (categories_boxes variable) are a tuple, with (categories, boxes). Each of these are a numpy array, with a length equivalent to the number of annotations present in the image. The categories have a width of 1, whereas the boxes have a width of 4.

In [ ]:
image, categories_boxes = train_ds.sample()

## Plot the image and its annotations

The following cell plots the image and its annotations, to confirm the dataset is loading correctly.

In [ ]:
fig, ax = plt.subplots(1)


# Create a caption map
cats = train_ds.coco.loadCats(train_ds.coco.getCatIds())
caption_map = {}
for cat in cats:
    if cat['id'] not in caption_map:
        caption_map[cat['id']] = cat['name']

ax.imshow(image)
cap_list = []
for n in range(len(categories_boxes[0])):
    box = categories_boxes[1][n]
    cat = categories_boxes[0][n]
    width = int(box[2])
    height = int(box[3])
    xy = (int(box[0]), int(box[1]))
    caption = caption_map[cat]
    cap_list.append(str(caption))
    rect = patches.Rectangle(xy, width, height, angle=0.0, alpha=0.5, label=caption)
    ax.add_patch(rect)
plt.title('Categories: {}'.format(' '.join(cap_list)))
plt.show()